# Initilize

### Libraries

In [1480]:
from pathlib import Path
from pprint import pprint
import json

### Variables & Config File

In [1481]:
# Variables

edl_path = 'reference\RTGA0715H.edl'
config_path = 'sources\config.json'
drop_frame = None
edit_hour_mark = ""  #variable set in detect_dropframe
dissolve_offset = ""  # variable set in detect_dropframe function.  this compensates for the fade in Vantage.  Vantage treats a fade where first frame is 0% opacity and the last frame is 100% opacity.  Avid treats the first & last frame as showing some of the video.  This cheats that look so first or last frame isn't blank.
framerate = 29.97  # Plugged into the cml_edit_code.
video_edits = []
audio_edits = []
media_files = []
cml_assembled = ""

# Load Variables from Confing File
# config_data = load_config(config_path)



In [1482]:
# Avid CML Code Snippets

cml_header_code = ""


cml_media_code = """
<Source identifier="{source_number}">
		<File location="{{$$Workflow_SupportingFilesPath}}\{media_file}" />
	</Source>
"""

cml_edit_code = """"
			<Video source="{source_number}" align="head" adjust="edge" offset="{{ {edit_in}-{dissolve_in_offset}-{edit_hour_mark} }}" filter="mute" >
				<Head>
					<Fade duration="{{ {dissolve_in}+{dissolve_in_offset} }}" />
				</Head>
				<Tail>
					<Fade duration="{{ {dissolve_out}+{dissolve_out_offset} }}" />	
					<Edit mode="duration" time="{{ {edit_out}+{dissolve_out_offset}-{edit_in} }}" />
				</Tail>
			</Video>
 """

In [1483]:
# #  Load Config File
# def load_config(config_path):
#     try:
#         with open(config_path, 'r') as c:
#             config = json.load(c)
#         return config
#     except FileNotFoundError:
#         print(f"Error: File not found at {config_path}")
#         return None
#     except json.JSONDecodeError:
#         print(f"Error: Could not decode JSON from {config_path}")
#         return None


# Open and Parse the EDL

### Open File

In [1484]:
# Opens EDL and extracts its contents into a single string.
def open_edl(edl_path):
    with open(edl_path, 'r') as edl_file:
        edl_contents = edl_file.read()
    edl_extract(edl_contents)    

In [1485]:
# Breasks EDL into a list of fields for each line
def edl_extract(edl_contents): 
    edl_lines = edl_contents.splitlines()

    edl_extracted = []

    for line in edl_lines:
        columns = line.split()
        edl_extracted.append(columns)
    
    
    simplify_edl(edl_extracted)


### Simplify EDL
Removes the extra columns and data from the EDL.  It also removes any lines where the in/out point match on the same line and no real edit is being made.

In [1486]:
def simplify_edl(edl):
	detect_dropframe(edl)
	simplified_edl = []

	for line in range(2, len(edl)):

		if edl[line][0] != "*" and edl[line][-1] != edl[line][-2]:  #if this doesn't have duplicate timecode for edits
			edits = {}
			dissolve_length = 0

			if edl[line][3] == "D":
				dissolve_length = int(edl[line][4])

			edits = {
				"media_type": edl[line][2][0],
				"edit_type": edl[line][3],
				"dissolve_length": dissolve_length,
				"edit_in" : edl[line][-2],
				"edit_out" : edl[line][-1]
				}

			simplified_edl.append(edits)  #  adds edits to the simplified EDL


		elif edl[line][0] == "*":  # lines with clip note
			edits = {
				"media_type" : edl[line][0],
				"media_file": edl[line][1]
			}

			simplified_edl.append(edits)  #  adds edl  to the simplified EDL 
	
	# pprint(simplified_edl)

	edl_split_video_audio(simplified_edl)	

### Split Video and Audio Edits

In [1487]:
#  Looks for relevant edit information:  Drop/Non-Drop Frame, Title Name, Edits
def edl_split_video_audio(edl):
    # Clears the edit list for parsing
    global video_edits
    global audio_edits

    edl_video = []
    edl_audio = []

    for line in range(0, len(edl)):
        #  Splits audio and video edits into seperate EDLs
        if edl[line]["media_type"] == "V":
            edl_video.append(edl[line])
            if line+1 < len(edl) and edl[line+1]["media_type"] == "*": 
                edl_video.append(edl[line+1])                

        if edl[line]["media_type"] == "A":
            edl_audio.append(edl[line])
            if line+1 < len(edl) and edl[line+1]["media_type"] == "*":
                edl_audio.append(edl[line+1])


    
    # Parses the edits
    if edl_video:
        video_edits = parse_edits(edl_video)
        

    if edl_audio:
        audio_edits = parse_edits(edl_audio)
        # edit_files = parse_media_files(edl_audio)

    # print("=========VIDEO=============")
    # pprint(edl_video)
    # print("=========AUDIO=============")
    # pprint(edl_audio)


### Parse Edits
Breaks out each edit out of the EDL.  It seperates the dissolves as their own clip.

In [1488]:
def parse_edits(edl):
    edit_list = []

    for line in range(0, len(edl)):  
        edits = {} 

        #  Audio Edits
        if line+1 < len(edl) and edl[line]["media_type"] == "A":
            #  Audio - Cut In - Cut Out 
                # (only option - no audio dissolves done in vantage  - any edits needs to be done on the media file before exporting to vantage)
            edits = {
                "type": "audio_edit",
                "dissolve_in" : 0,
                "dissolve_out" : 0,
                "edit_in" : edl[line]["edit_in"],
                "edit_out" : edl[line]["edit_out"],
                "media_file" : edl[line+1]["media_file"]            
            }
            edit_list.append(edits)

                

        # #  Video Edits
        if (edl[line]["media_type"] == "V"):

            #  DISSOLVE IN
            if (line+2 < len(edl) and
                edl[line]["edit_type"] == "D" and
                edl[line+1]["media_type"] != "*" and
                edl[line+2]["media_type"] == "*"           
                ): 

                edits = {
                    "type" : "dissolve_in",
                    "dissolve_in" : edl[line]["dissolve_length"],
                    "dissolve_out" : 0,
                    "edit_in" : edl[line]["edit_in"],
                    "edit_out" : edl[line]["edit_out"],
                    "media_file" : edl[line+2]["media_file"]            
                }

                edit_list.append(edits)

            # #  DISSOLVE OUT
            if (line+1 < len(edl) and
                edl[line]["edit_type"] == "D" and 
                edl[line+1]["media_type"] == "*"
                ): 
                     
                edits = {
                    "type" : "dissolve_out",
                    "dissolve_in" : 0,
                    "dissolve_out" : int(edl[line]["dissolve_length"]),
                    "edit_in" : edl[line]["edit_in"],
                    "edit_out" : edl[line]["edit_out"],
                    "media_file" : edl[line+1]["media_file"]            
                }
                  
                edit_list.append(edits)

            #  CUT IN CUT OUT
            if (line+1 < len(edl) and
                edl[line]["edit_type"] == "C" and 
                edl[line+1]["media_type"] == "*"
                ): 
                     
                edits = {
                    "type" : "cut_in_cut_out",
                    "dissolve_in" : 0,
                    "dissolve_out" : 0,
                    "edit_in" : edl[line]["edit_in"],
                    "edit_out" : edl[line]["edit_out"],
                    "media_file" : edl[line+1]["media_file"]            
                }
                  
                edit_list.append(edits)       

    consolodated_edits = consolodate_edits(edit_list)

    
    return consolodated_edits

### Timecode Tools

In [1489]:
def detect_dropframe(edl):
    global drop_frame
    global framerate
    global edit_hour_mark
    global dissolve_offset

     
    if edl[1][0] == "FCM:":
        if edl[1][1] == "DROP":
            drop_frame = True
            edit_hour_mark = f"01:00:00;00@{framerate}"
            dissolve_offset = f"00:00:00;01@{framerate}"

        else:
            drop_frame = False
            edit_hour_mark = f"01:00:00:00@{framerate}"
            dissolve_offset = f"00:00:00:01@{framerate}"


In [1490]:
# Convert timecode to dropframe
def format_timecode(timecode, framerate):
	global drop_frame

	if drop_frame == True:
		timecode=str(timecode.replace(":", ";").replace(";", ":", 2)) + "@" + str(framerate)
	
	else:
		timecode=str(timecode) + "@" + str(framerate)
	
	return timecode

In [1491]:
# Convert dissolves to timecode
def dissolves_timecode(dissolve, framerate):

	seconds = int(dissolve/round(framerate))
	frames = int(dissolve - (seconds*round(framerate)))

	if seconds < 10:
		seconds = f"0{seconds}"
	
	if frames < 10:
		frames = f"0{frames}"

	timecode = f"00:00:{seconds}:{frames}"

	return format_timecode(timecode, framerate)
	

###  Consolodate Edits
This adds the dissolves back into the the beginning/ending of applicable cuts-only edits.

In [1492]:
def consolodate_edits(edl):
	new_edit_list = []
	# global framerate
	
	for line in range(0, len(edl)):
		if edl[line]["type"] == "audio_edit":
			new_edits = {
                    "dissolve_in" : edl[line]["dissolve_in"],
                    "dissolve_out" : edl[line]["dissolve_out"],
                    "edit_in" : edl[line]["edit_in"],
                    "edit_out" : edl[line]["edit_out"],
                    "media_file" : edl[line]["media_file"]            
                }
                  
			new_edit_list.append(new_edits)


		elif edl[line]["type"] == "cut_in_cut_out":
			new_in = edl[line]["edit_in"]
			new_out = edl[line]["edit_out"]
			new_dissolve_in = 0
			new_dissolve_out = 0
			new_edits = {}

			#  Dissolve In - Dissolve Out
			for check in range(0, len(edl)):
				if (edl[check]["type"] == "dissolve_in" and
					edl[line]["edit_in"] == edl[check]["edit_out"] and
					edl[line]["media_file"] == edl[check]["media_file"]
					):

					new_in = edl[check]["edit_in"]
					new_dissolve_in = edl[check]["dissolve_in"]


				if (edl[check]["type"] == "dissolve_out" and
					edl[line]["edit_out"] == edl[check]["edit_in"] and
					edl[line]["media_file"] == edl[check]["media_file"]
					):

					new_out = edl[check]["edit_out"]
					new_dissolve_out = edl[check]["dissolve_out"]

			new_edits = {
                    "dissolve_in" : new_dissolve_in,
                    "dissolve_out" : new_dissolve_out,
                    "edit_in" : new_in,
                    "edit_out" : new_out,
                    "media_file" : edl[line]["media_file"]            
                }
                  
			new_edit_list.append(new_edits)
	
	# Extract the media files from EDL (either new blue titles or audio files)
	edit_files = parse_media_files(new_edit_list)

	return new_edit_list


### Extract Media Files
Extract a list of media files from the EDL

In [1493]:
# Extract the media files from EDL (either new blue titles or audio files)
def parse_media_files(edl):
    global media_files

    for media in edl:
        if media["media_file"] not in media_files:
            media_files.append(media["media_file"])


# Assemble CML

In [1494]:
# media_source_code = "Source # = {source_number}  Media source = {media_file}"

def cml_sources(media_files, cml_media_code):    
    sources_code = ""

    for line in range(len(media_files)):
        source_number = line + 1
        media_file = media_files[line]

        sources_code += cml_media_code.format(
            source_number=source_number,
            media_file=media_file
        )    
    
    return sources_code



In [ ]:
def cml_edits(media_files, edits):
	global framerate
	global dissolve_offset
	global edit_hour_mark
	edit_code = ""

	for line in range(len(edits)):
		source_number = ""		
		media_file = edits[line]['media_file']

		#  Finds the media source number for this edit
		for index, source in enumerate(media_files):
			if source == media_file:
				source_number = index+1
				break
		
		# Adjusts the dissolve offset to zero if the dissolve is zero...ie a cut
		if edits[line]['dissolve_in'] == 0:
			dissolve_in_offset = format_timecode("00:00:00:00", framerate)
		else:
			dissolve_in_offset = dissolve_offset

		if edits[line]['dissolve_out'] == 0:
			dissolve_out_offset = format_timecode("00:00:00:00", framerate)
		else:
			dissolve_out_offset = dissolve_offset
		
		# sends the variables to the edit code
		edit_code += cml_edit_code.format(
            source_number=source_number,
			edit_in = format_timecode(edits[line]['edit_in'],framerate),
			edit_out = format_timecode(edits[line]['edit_out'], framerate),
			dissolve_in = dissolves_timecode(edits[line]['dissolve_in'], framerate),
			dissolve_out = dissolves_timecode(edits[line]['dissolve_out'], framerate),
			dissolve_in_offset = dissolve_in_offset,
			dissolve_out_offset = dissolve_out_offset,
			edit_hour_mark = edit_hour_mark
        )  

		
		# print(f"{edits[line]['media_file']} - {source_number}")
		print(edit_code)

	# return edit_code
	



In [1496]:
def build_cml(media_files, video_edits, audio_edits):
	global cml_assembled
	global cml_media_code

	header = ""
	sources = cml_sources(media_files, cml_media_code)
	video_edits = cml_edits(media_files, video_edits)
	audio_edits = ""
	footer = ""

	print(video_edits)



# Execution 

In [1497]:


open_edl(edl_path)

# pprint(video_edits)

build_cml(media_files, video_edits, audio_edits)
# cml_edits(media_files, video_edits, audio_edits)






"
			<Video source="1" align="head" adjust="edge" offset="{ 01:01:31;20@29.97-00:00:00;00@29.97-01:00:00;00@29.97 }" filter="mute" >
				<Head>
					<Fade duration="{ 00:00:00;00@29.97+00:00:00;00@29.97 }" />
				</Head>
				<Tail>
					<Fade duration="{ 00:00:00;08@29.97+00:00:00;01@29.97 }" />	
					<Edit mode="duration" time="{ 01:04:21;28@29.97+00:00:00;01@29.97-01:01:31;20@29.97 }" />
				</Tail>
			</Video>
 
"
			<Video source="1" align="head" adjust="edge" offset="{ 01:01:31;20@29.97-00:00:00;00@29.97-01:00:00;00@29.97 }" filter="mute" >
				<Head>
					<Fade duration="{ 00:00:00;00@29.97+00:00:00;00@29.97 }" />
				</Head>
				<Tail>
					<Fade duration="{ 00:00:00;08@29.97+00:00:00;01@29.97 }" />	
					<Edit mode="duration" time="{ 01:04:21;28@29.97+00:00:00;01@29.97-01:01:31;20@29.97 }" />
				</Tail>
			</Video>
 "
			<Video source="1" align="head" adjust="edge" offset="{ 01:04:24;13@29.97-00:00:00;01@29.97-01:00:00;00@29.97 }" filter="mute" >
				<Head>
					<Fade duration